In [58]:
# Importing
import pandas as pd
import numpy as np
import plotly.express as px


path = "./cleaned_houses.csv"

df = pd.read_csv(path)

In [59]:
df.columns

Index(['district', 'price', 'state_construction', 'living_area', 'bedrooms',
       'bathrooms', 'facades', 'has_garden', 'kitchen', 'has_terrace',
       'has_attic', 'has_basement', 'epc', 'area_total'],
      dtype='object')

In [60]:
df ["price_sqm_total"] = df ["price"]/df ["living_area"]

In [61]:
df.describe()

,price,living_area,bedrooms,bathrooms,facades,has_garden,kitchen,has_terrace,has_attic,has_basement,epc,area_total,price_sqm_total
count,18520.000000,18520.000000,18520.000000,18520.000000,14679.000000,18520.000000,18520.000000,18520.000000,18520.000000,18520.000000,14928.000000,18520.000000,18520.000000
mean,343524.505400,169.423920,3.249460,1.198650,2.860004,0.384017,0.607289,0.516739,0.311771,0.387203,3.926313,467.492765,2066.267271
std,143344.246278,48.139696,0.897269,0.493063,0.787961,0.486375,0.488367,0.499733,0.463229,0.487124,1.808752,350.591360,761.012211
min,42500.000000,51.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,51.000000,242.857143
25%,239000.000000,135.000000,3.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,195.000000,1512.658228
50%,329000.000000,165.000000,3.000000,1.000000,3.000000,0.000000,1.000000,1.000000,0.000000,0.000000,4.000000,361.000000,2014.218009
75%,435000.000000,200.000000,4.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,650.000000,2535.757006
max,780000.000000,310.000000,10.000000,15.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000,1608.000000,7247.191011


In [62]:
def thousants_point(x: int) -> str:
    """Adds point to separate thousant in text.
    Args:
        x: number that will take the thousant point
        n: number"""
    return str('{:,}'.format(round(x)).replace(',','.'))

In [93]:
# GRAPH NUMBER OF HOUSES PER PRICE/(SQM)
# Sets up info to be added to plot (average) and its text
district = "Brussels"
prediction = 350000
area = 135

mask = df["district"] == district
subset_district = df[mask]

average_price = subset_district["price_sqm_total"].mean()
text_average_price = "Average €" + thousants_point(int(average_price)) + "/m²" 

# text prediction
price_sqm_predict = prediction/area
text_pred = "Your house €" + thousants_point(int(price_sqm_predict))  + "/m²"

# Starts setting up the histogram
fig = px.histogram(subset_district,  x="price_sqm_total", title=f"Number of houses versus price/m² in {district}", labels ={"price_sqm_total": "Price per living area (€/m²)"}) 
fig.add_vline(x=average_price,annotation=dict(font_size=15), line_dash = 'dash', line_color = 'firebrick', annotation_text= text_average_price, annotation_position="top")
fig.add_vline(x=price_sqm_predict,annotation=dict(font_size=15), line_dash = 'dash', line_color = 'green', annotation_text= text_pred, annotation_position="top left")
fig.add_annotation(x=average_price,
                    y= 0,
            text=f"Average of {district}",
            showarrow=True,
            arrowhead=1,
            xanchor="left", 
            ax = 20,
            ay = 5
            ),
            
fig.update_layout(bargap=0.2)

# Displays the p

In [64]:
district = "Brussels"

mask = df["district"] == district
df_bedrooms = df[mask]


df_bedrooms = df_bedrooms.groupby("bedrooms", as_index=False).count()

df_bedrooms = df_bedrooms [["bedrooms", "district"]]
df_bedrooms.rename(columns={"district": "Count", "bedrooms" : "Bedrooms"}, inplace= True)


df_bedrooms

,Bedrooms,Count
0,2,29
1,3,122
2,4,148
3,5,73
4,6,21
5,7,9
6,8,4


In [65]:
your_home = 2

pull = df_bedrooms.Bedrooms == your_home

pull = pull * 0.3 

pull

0    0.3
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
Name: Bedrooms, dtype: float64

In [66]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Pie(labels=df_bedrooms.Bedrooms, values=df_bedrooms.Count, textinfo='label+percent', hole=.4, pull=pull, showlegend=False)])

fig.update_layout(
    title_text=f"Distribution of number of bedrooms in {district}",
    # Add annotations in the center of the donut.
    annotations=[
        
        dict(
            text='Bedrooms', 
            # Square unit grid starting at bottom left of page
            x=0.5, y=0.5, 
            font_size=12,
            # Hide the arrow that points to the [x,y] coordinate
            showarrow=False
          
        )
    ]
)
fig.show()